## Homework

In this homework, we’ll deploy the ride duration model in batch mode.
Like in homework 1, we’ll use the Yellow Taxi Trip Records dataset.

You’ll find the starter code in the [homework](homework) directory.



In [20]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

#sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

## Q1. Notebook

We’ll start with the same notebook we ended up with in homework 1. We
cleaned it a little bit and kept only the scoring part. You can find the
initial notebook [here](homework/starter.ipynb).

Run this notebook for the February 2022 data.

What’s the standard deviation of the predicted duration for this
dataset?

-   5.28
-   10.28
-   15.28
-   20.28



In [4]:
def fetch(dataset_url: str) -> pd.DataFrame:
    print(dataset_url)
    # df = pd.read_csv(dataset_url, compression='gzip')
    df = pd.read_parquet(dataset_url)
    return df

In [5]:
def save_to_file(path):
    path = Path(f"{path}/trips_data_{months[0]:02}-{years[0]}_{months[-1]:02}-{years[-1]}.parquet")
    if not path.parent.is_dir():
        path.parent.mkdir(parents=True)
    path = Path(path).as_posix()
    df.to_parquet(path, compression="gzip")
    print(f"data saved to file: {path}")

In [9]:
colors = ["yellow"]
months = [2]
years = [2022]

In [7]:
df = pd.DataFrame()
for color in colors:
    for month in months:
        for year in years:
            url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{color}_tripdata_{year}-{month:02}.parquet"

            df = pd.concat([df, fetch(url)], ignore_index=True)

save_to_file("data")


https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
data saved to file: data/trips_data_02-2022_02-2022.parquet


In [17]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [18]:
df = pd.read_parquet(f"data/trips_data_{months[0]:02}-{years[0]}_{months[-1]:02}-{years[-1]}.parquet")

categorical = ['PULocationID', 'DOLocationID']

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

print(y_pred)

print(y_pred.std())


[18.52778307 23.06578208 33.68635854 ... 11.89045938 15.10268128
  9.46059157]
5.28140357655334


## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the
output.

First, let’s create an artificial `ride_id` column:

``` python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results.

Save it as parquet:

``` python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What’s the size of the output file?

-   28M
-   38M
-   48M
-   58M

**Note:** Make sure you use the snippet above for saving the file. It
should contain only these two columns. For this question, don’t change
the dtypes of the columns and use pyarrow, not fastparquet.



In [23]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [24]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    "output.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
import os

file_stats = os.stat("output.parquet")

print(file_stats.st_size)

59994935


## Q3. Creating the scoring script

Now let’s turn the notebook into a script.

Which command you need to execute for that?



jupyter nbconvert --to script starter.ipynb

## Q4. Virtual environment

Now let’s put everything into a virtual environment. We’ll use pipenv
for that.

Install all the required libraries. Pay attention to the Scikit-Learn
version: it should be `scikit-learn==1.2.2`.

After installing the libraries, pipenv creates two files: `Pipfile` and
`Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What’s the first hash for the Scikit-Learn dependency?



In [ ]:
pip install pipenv
pipenv --version
pipenv --python=3.9
pipenv install scikit-learn==1.2.2 pandas

## Q5. Parametrize the script

Let’s now make the script configurable via CLI. We’ll create two
parameters: year and month.

Run the script for March 2022.

What’s the mean predicted duration?

-   7.76
-   12.76
-   17.76
-   22.76

Hint: just add a print statement to your script.



## Q6. Docker container

Finally, we’ll package the script in the docker container. For that,
you’ll need to use a base image that we prepared.

This is how it looks like:

    FROM python:3.10.0-slim

    WORKDIR /app
    COPY [ "model2.bin", "model.bin" ]

(see [`homework/Dockerfile`](homework/Dockerfile))

We pushed it to
[`svizor/zoomcamp-model:mlops-3.10.0-slim`](https://hub.docker.com/layers/svizor/zoomcamp-model/mlops-3.10.0-slim/images/sha256-595bf690875f5b9075550b61c609be10f05e6915609ef4ea4ce9797116c99eff?context=repo),
which you should use as your base image.

That is, this is how your Dockerfile should start:

``` docker
FROM svizor/zoomcamp-model:mlops-3.10.0-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer and a
model. You will need to use them.

Important: don’t copy the model to the docker image. You will need to
use the pickle file already in the image.

Now run the script with docker. What’s the mean predicted duration for
April 2022?

-   7.92
-   12.83
-   17.92
-   22.83

## Bonus: upload the result to the cloud (Not graded)

Just printing the mean duration inside the docker image doesn’t seem
very practical. Typically, after creating the output file, we upload it
to the cloud storage.

Modify your code to upload the parquet file to S3/GCS/etc.

## Publishing the image to dockerhub

This is how we published the image to Docker hub:

``` bash
docker build -t mlops-zoomcamp-model:v1 .
docker tag mlops-zoomcamp-model:v1 svizor/zoomcamp-model:mlops-3.10.0-slim
docker push svizor/zoomcamp-model:mlops-3.10.0-slim
```